In [10]:
import torch
import torchvision
from torchvision.transforms import functional as TF
import numpy as np
import cv2
import os
from PIL import Image

def load_detection_model():
    # Load a pre-trained Faster R-CNN model with a ResNet-50 backbone
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

def analyze_image(model, image_path, output_directory):
    # Open and convert the image to RGB
    img = Image.open(image_path).convert("RGB")
    img_tensor = TF.to_tensor(img).unsqueeze(0)

    with torch.no_grad():
        results = model(img_tensor)[0]

    # Convert PIL image to OpenCV format
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    # Number of detected objects and masks
    num_detected = len(results["boxes"])
    num_masks = num_detected  # One mask per bounding box

    # Draw bounding boxes and labels on the image
    for idx in range(num_detected):
        bbox = results["boxes"][idx].cpu().numpy().astype(int)
        confidence = results["scores"][idx].item()

        if confidence > 0.5:  # Threshold for filtering detections
            label = f"Confidence: {confidence:.2f}"
            cv2.rectangle(img_cv, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
            cv2.putText(img_cv, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.putText(img_cv, f"Detected Items: {num_detected}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(img_cv, f"Masks Available: {num_masks}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)

    # Create output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    output_file = os.path.join(output_directory, os.path.basename(image_path))
    cv2.imwrite(output_file, img_cv)

    return num_detected, num_masks

def process_images_in_directory(model, input_directory, output_directory):
    for file_name in os.listdir(input_directory):
        if file_name.lower().endswith(".jpg"):
            full_path = os.path.join(input_directory, file_name)
            detected_items, masks_count = analyze_image(model, full_path, output_directory)
            print(f"File: {file_name} - Detected Items: {detected_items}, Masks Count: {masks_count}")

if __name__ == "__main__":
    # Initialize the Faster R-CNN model
    detection_model = load_detection_model()

    screws_and_bolts_path = '/content/drive/MyDrive/ScrewAndBolt_20240713'
    screws_and_bolts_output_path = 'cnn_AI_output_ScrewAndBolt'

    print("Processing Screws and Bolts dataset...")
    process_images_in_directory(detection_model, screws_and_bolts_path, screws_and_bolts_output_path)



Processing Screws and Bolts dataset...
File: 20240713_193135.jpg - Detected Items: 12, Masks Count: 12
File: 20240713_193650.jpg - Detected Items: 10, Masks Count: 10
File: 20240713_193659.jpg - Detected Items: 12, Masks Count: 12
File: 20240713_192951.jpg - Detected Items: 13, Masks Count: 13
File: 20240713_193831.jpg - Detected Items: 33, Masks Count: 33
File: 20240713_193839.jpg - Detected Items: 40, Masks Count: 40
File: 20240713_193907.jpg - Detected Items: 100, Masks Count: 100
File: 20240713_194200.jpg - Detected Items: 65, Masks Count: 65
File: 20240713_194206.jpg - Detected Items: 90, Masks Count: 90
File: 20240713_194215.jpg - Detected Items: 66, Masks Count: 66
File: 20240713_194232.jpg - Detected Items: 71, Masks Count: 71
File: 20240713_194256.jpg - Detected Items: 100, Masks Count: 100
File: 20240713_194316.jpg - Detected Items: 96, Masks Count: 96
File: 20240713_194541.jpg - Detected Items: 6, Masks Count: 6
File: 20240713_194551.jpg - Detected Items: 4, Masks Count: 4
F